## 🪄 Install `raga-testing-platform` library


### Put your `ACCESS KEY & SECRET KEY`

---


In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"
HOST = "https://backend.platform.raga.ai"

### Import All raga lib from raga module


In [ ]:
from raga import *
import datetime

## #`Object Detection Dataset v2 (Dummy patients with KneeCoil)`


In [ ]:
run_name = f"tt"

### Drift Detection Test

---


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

train_dataset_name = "philip_train_dataset_v1"
test_dataset_name = "kneecoil_april"

rules = DriftDetectionRules()
rules.add(type="single_class_anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold= 44)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name=f"philip_drift_v1",
                                           train_dataset_name=train_dataset_name,
                                           field_dataset_name=test_dataset_name,
                                           train_embed_col_name="ImageEmbedding",
                                           field_embed_col_name = "ImageVectorsM1",
                                           level = "image",
                                           rules = rules)


test_session.add(edge_case_detection)

test_session.run()

## ###Near Duplicates Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
rules = LQRules()
rules.add(metric="similarity_score", metric_threshold=0.99)

train_dataset_name = "philip_train_dataset_v1"
# test_dataset_name = "philip_val_dataset_v1"

edge_case_detection = nearest_duplicate(test_session=test_session,
                                          dataset_name = train_dataset_name,
                                          test_name = "Near-Duplicates-v1",
                                          type = "near_duplicates",
                                          output_type="near_duplicates",
                                          embed_col_name="ImageEmbedding",
                                          rules=rules)

test_session.add(edge_case_detection)

test_session.run()

## ###Outlier Detection Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
train_dataset_name = "philip_train_dataset_v1"
# test_dataset_name = "philip_val_dataset_v1"

rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=18.57)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name="Outlier_detection",
                                           dataset_name= train_dataset_name,
                                           embed_col_name="ImageEmbedding",
                                           output_type = "outlier_detection",
                                           rules = rules)


test_session.add(edge_case_detection)

test_session.run()

## ###Failure Mode Analysis Test - Clustering


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
dataset_name = "kneecoil_april"

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.3, metric_threshold=0.7, iou_threshold=0.5, label="ALL")
rules.add(metric="F1Score", conf_threshold=0.3, metric_threshold=0.7, iou_threshold=0.5, label="ALL")
rules.add(metric="Recall", conf_threshold=0.3, metric_threshold=0.7, iou_threshold=0.5, label="ALL")

cls_default = clustering(test_session=test_session, dataset_name = dataset_name, method="k-means", embedding_col="ImageVectorsM1", level="image", args= {"numOfClusters": 8}, force=True)

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = f"FMA_Clustering",
                                            model = "ModelA",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="object_detection",
                                            type="embedding",
                                            clustering=cls_default
                                            )

test_session.add(edge_case_detection)

test_session.run()

###Failure Mode Analysis Test - Metadata

---


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

dataset_name = "kneecoil_april"

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.4, metric_threshold=0.6, iou_threshold=0.4, label="ALL")


edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "fma_meta1",
                                            model = "ModelA",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="object_detection",
                                            type="metadata",
                                            aggregation_level=['brightness']
                                            )

test_session.add(edge_case_detection)

test_session.run()

## ###Scenario Imbalance Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
augmented_dataset_name = "kneecoil_april"
# test_dataset_name = "philip_val_dataset_v1"

rules = SBRules()
rules.add(metric="js_divergence", ideal_distribution="uniform", metric_threshold=0.1)
rules.add(metric="chi_squared_test", ideal_distribution="uniform", metric_threshold=0.1)

edge_case_detection = scenario_imbalance(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = "Scenario_Imbalance4",
                                            type = "scenario_imbalance",
                                            output_type="metadata",
                                            rules = rules,
                                            aggregationLevels=["brightness"]
                                             )

test_session.add(edge_case_detection)

test_session.run()

## ###Labeling Quality Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
dataset_name = "philips_field_data_mistakes"

rules = LQRules()
rules.add(metric="mistake_score", label=["ALL"], metric_threshold= 0.266)


edge_case_detection = labelling_quality_test(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "Labeling Quality Test",
                                            type = "labelling_consistency",
                                            output_type="object_detection",
                                            mistake_score_col_name = "MistakeScore",
                                            gt="GT",
                                            # model="GT",
                                            rules = rules)
test_session.add(edge_case_detection)

test_session.run()

## #`Object Detection Dataset v1 (Various backgrounds with KneeCoil)`


In [ ]:
run_name2 = f"tt1"

### Data Drift Test

---


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

train_dataset_name = "philip_train_dataset_v1"
test_dataset_name = "philip_val_dataset_v1"

rules = DriftDetectionRules()
rules.add(type="single_class_anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold= 42)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name=f"philip_drift_v1",
                                           train_dataset_name=train_dataset_name,
                                           field_dataset_name=test_dataset_name,
                                           train_embed_col_name="ImageEmbedding",
                                           field_embed_col_name = "ImageEmbedding",
                                           level = "image",
                                           rules = rules)


test_session.add(edge_case_detection)

test_session.run()


## ###Active Learning Test


In [ ]:
budget = 2000

test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

train_dataset_name = "philip_train_dataset_v1"
# test_dataset_name = "philip_val_dataset_v1"

edge_case_detection = active_learning(test_session=test_session,
                                      dataset_name = train_dataset_name,
                                      test_name = "active_learniing_train_v1",
                                      type = "active_learning",
                                      output_type="curated_dataset",
                                      embed_col_name="ImageEmbedding",
                                      budget=budget)

test_session.add(edge_case_detection)

test_session.run()

## ###Near Duplicates Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
rules = LQRules()
rules.add(metric="similarity_score", metric_threshold=0.94)

train_dataset_name = "philip_train_dataset_v1"
# test_dataset_name = "philip_val_dataset_v1"

edge_case_detection = nearest_duplicate(test_session=test_session,
                                          dataset_name = train_dataset_name,
                                          test_name = "Near-Duplicates-v1",
                                          type = "near_duplicates",
                                          output_type="near_duplicates",
                                          embed_col_name="ImageEmbedding",
                                          rules=rules)

test_session.add(edge_case_detection)

test_session.run()

###Data Leakage Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

train_dataset_name = "philip_train_dataset_v1"
test_dataset_name = "philip_val_dataset_v1"

rules = LQRules()
rules.add(metric = 'overlapping_samples', metric_threshold = 0.95)


edge_case_detection = data_leakage_test(test_session=test_session,
                                           test_name="dataleakage_philips_v1",
                                           train_dataset_name=train_dataset_name,
                                           dataset_name=test_dataset_name,
                                           type = "data_leakage",
                                           output_type="image_data",
                                           train_embed_col_name="ImageEmbedding",
                                           embed_col_name = "ImageEmbedding",
                                           rules = rules)

test_session.add(edge_case_detection)

test_session.run()

## ###Outlier Detection


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
train_dataset_name = "philip_train_dataset_v1"
# test_dataset_name = "philip_val_dataset_v1"

rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=40)

edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name="Outlier_detection",
                                           dataset_name= train_dataset_name,
                                           embed_col_name="ImageEmbedding",
                                           output_type = "outlier_detection",
                                           rules = rules)


test_session.add(edge_case_detection)

test_session.run()

## ###Failure Mode Analysis - Clustering


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
augmented_dataset_name = "train_augmentation"
# test_dataset_name = "philip_val_dataset_v1"

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.4, metric_threshold=0.7, iou_threshold=0.4, label="ALL")
rules.add(metric="F1Score", conf_threshold=0.4, metric_threshold=0.7, iou_threshold=0.4, label="ALL")
rules.add(metric="Recall", conf_threshold=0.4, metric_threshold=0.7, iou_threshold=0.4, label="ALL")

cls_default = clustering(test_session=test_session, dataset_name = dataset_name, method="k-means", embedding_col="ImageVectorsM1", level="image", args= {"numOfClusters": 8}, force=True)

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = f"FMA_Clustering",
                                            model = "ModelA",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="object_detection",
                                            type="embedding",
                                            clustering=cls_default
                                            )

test_session.add(edge_case_detection)

test_session.run()

## ###Failure Mode Analysis - Metadata


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

augmented_dataset_name = "train_augmentation"

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.4, metric_threshold=0.5, iou_threshold=0.4, label="ALL")


edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = "fma_meta1",
                                            model = "ModelA",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="object_detection",
                                            type="metadata",
                                            aggregation_level=['Augmentation']
                                            )

test_session.add(edge_case_detection)

test_session.run()

## ###Scenario Imbalance


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

augmented_dataset_name = "train_augmentation"

rules = SBRules()
rules.add(metric="js_divergence", ideal_distribution="uniform", metric_threshold=0.1)
rules.add(metric="chi_squared_test", ideal_distribution="uniform", metric_threshold=0.1)

edge_case_detection = scenario_imbalance(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = "Scenario_Imbalance4",
                                            type = "scenario_imbalance",
                                            output_type="metadata",
                                            rules = rules,
                                            aggregationLevels=["augmentation"]
                                             )

test_session.add(edge_case_detection)

test_session.run()

## ###Labeling Quality Test


In [ ]:
test_session = TestSession(
    project_name="KneeCoil Detection",
    run_name=run_name2,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)
dataset_name = "train13"

rules = LQRules()
rules.add(metric="mistake_score", label=["ALL"], metric_threshold=0.93)


edge_case_detection = labelling_quality_test(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "Labeling Quality Test",
                                            type = "labelling_consistency",
                                            output_type="object_detection",
                                            mistake_score_col_name = "MistakeScore",
                                            gt="GT",
                                            # model="GT",
                                            rules = rules)
test_session.add(edge_case_detection)

test_session.run()